# 词袋嵌入

In [1]:
import csv
import math
import re
from abc import abstractmethod, ABC
from collections import Counter

import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### IMDB数据集

In [8]:
class IMDBDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = [list(set(index)) for index in self.tokens[:-10]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[:-10]]

    def eval(self):
        self.features = [list(set(index)) for index in self.tokens[-10:]]
        self.labels = [0 if index == "negative" else 1 for index in self.sentiments[-10:]]

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def estimate(self, predictions):
        count = 0
        for i in range(len(predictions)):
            if np.abs(predictions[i].data - self.labels[i]) < 0.5:
                count += 1
        return count / len(predictions)

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [10]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 嵌入层

In [11]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.randn(embedding_size, vocabulary_size) * np.sqrt(2 / vocabulary_size))

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if type(self.weight.grad) is not np.ndarray:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

    def __str__(self):
        return f'Embedding[weight{self.weight.data.shape}; vocabulary={self.vocabulary_size}; embedding={self.embedding_size}]'

### ReLU激活函数

In [12]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'ReLU[]'

### Sigmoid激活函数

In [13]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Sigmoid[]'

### 损失函数（二元交叉熵）

In [14]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped) + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped) * len(y.data))

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### 优化器（随机梯度下降）

In [15]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经元网络模型

In [16]:
class RNNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)

                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        predictions = []
        for i in range(len(dataset)):
            feature, label = dataset[i]
            predictions.append(self.layer(feature))
        return predictions

## 设置

### 学习率

In [17]:
LEARNING_RATE = 0.01

### 轮次

In [18]:
EPOCHS = 100

## 训练

### 迭代

In [19]:
dataset = IMDBDataset('tinyimdb.csv')
layer = Sequential([Embedding(len(dataset.vocabulary), 64),
                    ReLU(),
                    Linear(64, 16),
                    ReLU(),
                    Linear(16, 1),
                    Sigmoid()])
loss = BCELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = RNNModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Embedding[weight(64, 86); vocabulary=86; embedding=64]
ReLU[]
Linear[weight(16, 64); bias(16,)]
ReLU[]
Linear[weight(1, 16); bias(1,)]
Sigmoid[]


## 验证

### 测试

In [20]:
predictions = model.test(dataset)
print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 1.0


### 近义词

In [21]:
def similar(dataset, layer, target='excellent'):
    target_index = dataset.word2index[target]
    scores = Counter()

    for word, index in dataset.word2index.items():
        raw_diff = layer.layers[0].weight.data.T[index] - layer.layers[0].weight.data.T[target_index]
        squared_diff = raw_diff ** 2
        scores[word] = -math.sqrt(sum(squared_diff))

    return scores.most_common(10)

print(similar(dataset, layer, target='excellent'))
print(similar(dataset, layer, target='terrible'))

[('excellent', -0.0), ('loved', -1.5319985233340763), ('with', -1.5632639897677165), ('love', -1.6116204598449295), ('our', -1.634810547813047), ('us', -1.6589551282584118), ('action', -1.6604934751192915), ('i', -1.6667580752365656), ('director', -1.669429274379996), ('amazing', -1.6695803418606885)]
[('terrible', -0.0), ('hate', -1.6149914803074807), ('sound', -1.6301151379060839), ('and', -1.642381180527386), ('awful', -1.6605246176684627), ('worst', -1.679045624417362), ('scene', -1.6970341442331274), ('see', -1.7162006220134063), ('our', -1.719311905151161), ('soundtrack', -1.7349640436993041)]
